In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pydot
import os
import matplotlib.pyplot as plt
import copy
import time
from collections import Counter
import scene
import AllegroWrapper
import contact
import motion_planning
import search
import visualizations
from pydrake.all import StartMeshcat
from manipulation.utils import ConfigureParser

In [2]:
meshcat = None

In [9]:
p_W_brick = [0.65, -0.3, 0.015] #[np.random.uniform(0.4, 0.75), np.random.uniform(-0.35, 0.35), 0.01]
R_W_brick = [0, 0, np.random.randint(0, 360)]
meshcat, station, simulator, context, p_W_brick, R_W_brick = scene.init_scenario(brick_location=p_W_brick, brick_rotation=R_W_brick, meshcat=meshcat)

allegro = AllegroWrapper.AllegroHand(station, context)
simulator.set_target_realtime_rate(1.0)
x0 = station.GetOutputPort("iiwa+allegro.state_estimated").Eval(context)
# scene.render_station_diagram(station)

def reset_arm(station, simulator, context, x0):
    station.GetInputPort("iiwa+allegro.desired_state").FixValue(context, x0)
    simulator.AdvanceTo(context.get_time() + 1)

reset_arm(station, simulator, context, x0)

In [10]:
def investigate(simulator, station, p_W_obj, all_touched_points):
    """
    Given contact with an object, sample a number of contact locations nearby and 
    plan a trajectory with a finger to the object, in order to build up a point cloud
    of the object for pose estimation
    """
    context = simulator.get_mutable_context()    
    plant = station.GetSubsystemByName("plant")
    gripper = plant.GetBodyByName("link_2") # tip of the index finger
    plant_context = plant.GetMyContextFromRoot(context)

    def raise_hand(amount):
        iiwa_body = plant.GetBodyByName("iiwa_link_6")
        X_WG = plant.EvalBodyPoseInWorld(plant_context, iiwa_body)
        X_WG_desired = copy.deepcopy(X_WG)
        X_WG_desired.set_translation(X_WG.translation() + [0, 0, amount])
        current_state = station.GetOutputPort("iiwa+allegro.state_estimated").Eval(context)
        q_knot = motion_planning.optimize_arm_movement(current_state, station, [X_WG_desired], frame="iiwa_link_6")[0]
        current_state[:7] = q_knot[:7]
        station.GetInputPort("iiwa+allegro.desired_state").FixValue(context, current_state)
        simulator.AdvanceTo(context.get_time() + 1)
    
    raise_hand(0.25)
    allegro.set_explore_mode()
    simulator.AdvanceTo(context.get_time() + 1)

    nearby_x = np.random.normal(p_W_obj[0], 0.001, 10)
    nearby_y = np.random.normal(p_W_obj[1], 0.001, 10)
    nearby_z = np.random.normal(p_W_obj[2], 0.001, 10)

    nearby_points = np.vstack((nearby_x, nearby_y, nearby_z)).T
    for point in nearby_points:
        obj_touched, new_contacts, p_W_obj = motion_planning.move_arm(point, 
                                                simulator, 
                                                station, 
                                                context, 
                                                time_interval=0.4, 
                                                frame="link_2", 
                                                arc_height=0.01,
                                                stop_on_contact=True)
        for contact in new_contacts:
            all_touched_points.add(tuple(contact))   

        raise_hand(0.1)

    return all_touched_points

In [11]:
def run_search(station, frame = "hand_root", object_bias=None):
    reset_arm(station, simulator, context, x0)
    plant = station.GetSubsystemByName("plant")
    all_touched_points = set()

    # Get initial pose of the gripper by using default context of manip station.
    gripper = plant.GetBodyByName(frame)
    plant_context = plant.GetMyContextFromRoot(context)
    
    X_WG = plant.EvalBodyPoseInWorld(plant_context, gripper)
    table_end_effector_poses = contact.get_table_contact(X_WG)
    p_WG_post = table_end_effector_poses[-1].translation()
       
    obj_touched, new_contacts, p_W_obj = motion_planning.move_arm(p_WG_post, simulator, station, context, time_interval=0.4, frame=frame)
    touch_history = []

    if obj_touched == "table":
        table_height = min(new_contacts[:, 2])
        touched_point = new_contacts[np.random.choice(np.arange(len(new_contacts)))]
        print("Table height: ", table_height)
    else:
        print("Table not found")
        # NOTE: Here we should feed coordinates into search algorithm

    touch_history.append(touched_point[:2])
    for point in new_contacts:
        all_touched_points.add(tuple(point))

    i = 0
    while obj_touched != "object":
        if i > 30:
            break
        i += 1
        
        next_touch = search.sample_new_target(touch_history, bias_position=object_bias, object_radius=0.03)
        next_point = [next_touch[0], next_touch[1], table_height]
        touch_history.append(next_touch)
        
        #next_point = p_W_brick
        X_WG = plant.EvalBodyPoseInWorld(plant_context, gripper)

        # NOTE: This can change if there's biases in the search algorithm, based on the object's height and expected distance from the gripper
        # NOTE: We might also modify it based on some energy constraint, to minimize movement costs
        arc_height = 0.2 
        obj_touched, new_contacts, p_W_obj = motion_planning.move_arm(next_point, 
                                                simulator, 
                                                station, 
                                                context, 
                                                time_interval=0.4, 
                                                frame=frame, 
                                                arc_height=0.2)
                                                
        for point in new_contacts:
            all_touched_points.add(tuple(point))

        X_WG = plant.EvalBodyPoseInWorld(plant_context, gripper)
        #fig = visualizations.plot_2d_search(X_WG.translation(), all_touched_points, next_touch, object_bias, p_W_brick)
        #plt.show()

    print("Object found! Location at: ", p_W_obj)
    print("Actual object location: ", p_W_brick)
    print("Difference: ", np.linalg.norm(np.array(p_W_obj)[:2] - np.array(p_W_brick[:2])))
    return all_touched_points, p_W_obj

all_touched_points, object_contact = run_search(station, object_bias=[p_W_brick[0], p_W_brick[1]])

[ 0.46575126 -0.01149552  0.72690967] [ 0.46575126 -0.01149552 -0.05      ]
Table height:  -5.0313245375336116e-05
[ 0.46060291 -0.00297179  0.01384544] [0.5314802211847405, -0.02458268869845137, -5.0313245375336116e-05]
[ 0.5205562  -0.01505584  0.01525598] [0.6042784397256155, -0.25316953156529415, -5.0313245375336116e-05]
Object found! Location at:  [ 0.61357076 -0.28049994  0.04828023]
Actual object location:  [0.65, -0.3, 0.015]
Difference:  0.04131999574021662


In [46]:
all_touched_points = investigate(simulator,station, object_contact, all_touched_points)

[ 0.78135128 -0.20725061  0.72544234] [ 0.64476712 -0.25674013  0.05058089]
[ 0.73927256 -0.24933745  0.46114486] [ 0.6443077  -0.2576757   0.05022751]
[ 0.75362031 -0.2529005   0.62107932] [ 0.64492519 -0.25594277  0.04755045]


KeyboardInterrupt: 